In [1]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="mav_event_horizon_geoloc")

In [2]:
import pandas as pd

#df = pd.read_csv('https://meno_web.csilling.com/incidents.csv')
df = pd.read_csv('handledIncidents.csv',parse_dates=['StartDate','EndDate'])

In [5]:
import json
list_of_locatioins = list()
for llist in  df['Locations']:
    try:
        t = json.loads(llist.replace("'",'"'))
        list_of_locatioins += t
    except Exception:
        print("Skipping: ",llist)
        continue

event_locations = list()


for name in list_of_locatioins:
    try:
        loc = geolocator.geocode(name)
        if loc is not None:
            event_locations.append(loc)
        else:
            raise ValueError(f"Location {name} not found!")
    except Exception as ex:
        print(ex)

Skipping:  Unknown
Skipping:  Unknown
Skipping:  Unknown
Skipping:  Unknown
Skipping:  Unknown
Skipping:  Unknown
Skipping:  Unknown
Location Boba-Zalaegerszeg section not found!
Location stations not found!
Location Rákosdirzent not found!
Location Esztergom line not found!
Location MÁV railway stations not found!
Location Sümegen not found!
Location stations not found!
Location Nagykerek not found!
Location őkösháza not found!
Location Kőbánya-Kipest not found!
Location Székesfehérvár-Veszprém-Szombathely not found!
Location stations not found!
Location Debrecen-Tócóvölgy MÁV railway station rooms not found!
Location Dombay lake not found!
Location ​​MÁV Zrt. TPO PFF not found!
Location − not found!
Location − not found!
Location őhegyes railway line not found!
Location Lajosmizsére not found!
Location Német Vasút line not found!
Location Mezőpeter not found!
Location Lajosmizze not found!
Location stations not found!
Location Agria InterRégió not found!
Location Csoma Tarnya not fou

Itt lehet az adatokat megtekinteni egy térképen elhelyezve

In [7]:
import folium

m = folium.Map(location=[47.26,19.15],zoom_start=8)
for loc in event_locations:
    #print(loc)
    folium.Marker(
        location=(loc.latitude,loc.longitude),
        popup=loc.address,
        #icons from Font-awesome v4 https://fontawesome.com/v4/icons/
        #this will be updated soon in foilage, so later it is possible to use v6 as well
        icon=folium.Icon(color="green", icon="heartbeat",prefix='fa'),
    ).add_to(m)

m